In [ ]:
%load_ext autoreload
%autoreload 2

%matplotlib inline

## Install libraries

```bash
conda create -n edu4 python=3.11 jupyter matplotlib
```

```bash 
! pip install -U -r requirements.txt
```

```bash
! pip install -U numpy
! pip install -U scikit-learn
```

In [ ]:
! ls

In [ ]:
! pip install -U -r requirements.txt

## Update repository

In [ ]:
! git pull

## Add import path

In [ ]:
import os
import sys
import gc

In [ ]:
module_path = os.path.abspath(os.path.join('..'))
if module_path not in sys.path:
    sys.path.append(module_path)

In [ ]:
del module_path

## Organize imports

In [ ]:
import multiprocessing
from pathlib import Path

In [ ]:
import seaborn as sns

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import sklearn
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import (StandardScaler, PolynomialFeatures, 
                                   SplineTransformer)
from sklearn.feature_selection import SequentialFeatureSelector as SFS
from sklearn.discriminant_analysis import (LinearDiscriminantAnalysis, 
                                           QuadraticDiscriminantAnalysis)
from sklearn.linear_model import LinearRegression, Ridge, Lasso, LogisticRegression
from sklearn.metrics import (mean_absolute_error, mean_squared_error, 
                             classification_report, confusion_matrix)

#### Number of CPU cores

In [ ]:
workers = multiprocessing.cpu_count()
workers

## Initialize path

In [ ]:
DATA = Path('data')
PATH = DATA / 'log_regr_lda_qda_np'
LR_PATH = DATA / 'linear_regression'
PUMPKIN_DIR = PATH / 'Pumpkin_Seeds_Dataset'
IRIS_DIR = PATH / 'iris'
PUMPKIN_DIR.mkdir(exist_ok=True, parents=True)
IRIS_DIR.mkdir(exist_ok=True, parents=True)

In [ ]:
! ls

## Load IRIS dataset

In [ ]:
SEED = 2022

In [ ]:
iris_url = 'https://www.kaggle.com/datasets/uciml/iris/download?datasetVersionNumber=2'

#### Load dataset

In [ ]:
! ls {IRIS_DIR}

In [ ]:
df = pd.read_csv(IRIS_DIR / 'Iris.csv')

In [ ]:
df

In [ ]:
df['Species'].value_counts()

In [ ]:
y = df['Species']
X = df.drop(columns=['Id', 'Species'], axis=1)
X.shape, y.shape, df['Species'].value_counts()

```python
# #define predictor and response variables
X = df[['s_length', 's_width', 'p_length', 'p_width']]
y = df['species']
X.shape, y.shape
```

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=SEED)

## Generate polynomial features

In [ ]:
poly = PolynomialFeatures(degree=2)

In [ ]:
X_train_pl = poly.fit_transform(X_train)
X_test_pl = poly.transform(X_test)

In [ ]:
feature_names_pl = poly.get_feature_names_out(input_features=X_train.columns)
feature_names_pl

In [ ]:
X_train.shape, X_test.shape, X_train_pl.shape, X_test_pl.shape

## Initialize the model

In [ ]:
lda = LinearDiscriminantAnalysis()

## Train model with polynomial features

In [ ]:
lda_pl = lda.fit(X_train_pl, y_train)

In [ ]:
y_pred_pl = lda_pl.predict(X_test_pl)

In [ ]:
cr = classification_report(y_test, y_pred_pl)
print(cr)

#### Forward selection

In [ ]:
X_train_pl.shape

In [ ]:
sfs_forward = SFS(lda, 
                  n_features_to_select=2, 
                  direction='forward',
                  cv=10,
                  n_jobs=workers)

sfs_forward = sfs_forward.fit(X_train_pl, y_train)
sfs_forward

In [ ]:
X_train_fw = sfs_forward.transform(X_train)
X_test_fw = sfs_forward.transform(X_test)
X_train_fw.shape

In [ ]:
X_train.columns

In [ ]:
sfs_forward.get_feature_names_out(X_train.columns)

In [ ]:
lda_fw = LinearDiscriminantAnalysis()

In [ ]:
lda_fw = lda_fw.fit(X_train_fw, y_train)

In [ ]:
y_pred_fw = lda_fw.predict(X_test_fw)

In [ ]:
cr = classification_report(y_test, y_pred_fw)
print(cr)

#### Backward selection\

In [ ]:
lda = LinearDiscriminantAnalysis()

In [ ]:
X_train_pl.shape

In [ ]:
sfs_backward = SFS(lda, 
                  n_features_to_select=2, 
                  direction='backward',
                  cv=10,
                  n_jobs=workers)

sfs_backward = sfs_backward.fit(X_train_pl, y_train)
sfs_backward

In [ ]:
X_train_bw = sfs_backward.transform(X_train_pl)
X_test_bw = sfs_backward.transform(X_test_pl)
X_train_bw.shape

In [ ]:
sfs_backward.get_feature_names_out(input_features=feature_names_pl)

In [ ]:
lda_bw = LinearDiscriminantAnalysis()

In [ ]:
lda_bw = lda_bw.fit(X_train_bw, y_train)

In [ ]:
y_pred_bw = lda_bw.predict(X_test_bw)

In [ ]:
cr = classification_report(y_test, y_pred_bw)
print(cr)

## Ridge and Lasso regression

In [ ]:
pertol_consumption_path = LR_PATH / 'petrol_consumption.csv'

In [ ]:
df = pd.read_csv(pertol_consumption_path)
df

In [ ]:
df.shape

In [ ]:
print(df.describe().round(2).T)

#### Prepare dataset

In [ ]:
y = df['Petrol_Consumption']
X = df[['Average_income', 'Paved_Highways',
       'Population_Driver_licence(%)', 'Petrol_tax']]

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, 
                                                    test_size=0.2, 
                                                    random_state=SEED)

## Generate Polynomial features

In [ ]:
poly = PolynomialFeatures(degree=4)

In [ ]:
X_train_pl = poly.fit_transform(X_train)
X_test_pl = poly.transform(X_test)

In [ ]:
feature_names_pl = poly.get_feature_names_out(input_features=X_train.columns)
feature_names_pl

In [ ]:
X_train.shape, X_train_pl.shape

In [ ]:
std_scaller = StandardScaler()
X_train_pl = std_scaller.fit_transform(X_train_pl)
X_test_pl = std_scaller.transform(X_test_pl)

#### Ridge

In [ ]:
ridge_rg = Ridge(alpha=100.08)
ridge_rg.fit(X_train_pl, y_train)

In [ ]:
ridge_rg.intercept_

In [ ]:
ridge_rg.coef_

In [ ]:
model_coefficients = ridge_rg.coef_

coefficients_df = pd.DataFrame(data = model_coefficients, 
                              index = feature_names_pl, 
                              columns = ['Coefficient value'])
print(coefficients_df)

#### Inference with the model

In [ ]:
y_pred_pl = ridge_rg.predict(X_test_pl)
y_pred_pl

In [ ]:
results = pd.DataFrame({'Actual': y_test, 'Predicted': y_pred_pl})
print(results)

#### Evaluate the model

In [ ]:
mae = mean_absolute_error(y_test, y_pred_pl)
mse = mean_squared_error(y_test, y_pred_pl)
rmse = np.sqrt(mse)

print(f'Mean absolute error: {mae:.2f}')
print(f'Mean squared error: {mse:.2f}')
print(f'Root mean squared error: {rmse:.2f}')

In [ ]:
actual_minus_predicted = sum((y_test - y_pred_pl)**2)
actual_minus_actual_mean = sum((y_test - y_test.mean())**2)
r2 = 1 - actual_minus_predicted/actual_minus_actual_mean
print('R²:', r2)

In [ ]:
ridge_rg.score(X_test_pl, y_test)

In [ ]:
ridge_rg.score(X_train_pl, y_train)

#### Lasso

In [ ]:
lasso_rg = Lasso(alpha=1.8)
lasso_rg.fit(X_train_pl, y_train)

In [ ]:
lasso_rg.intercept_

In [ ]:
lasso_rg.coef_

In [ ]:
model_coefficients = lasso_rg.coef_

coefficients_df = pd.DataFrame(data = model_coefficients, 
                              index = feature_names_pl, 
                              columns = ['Coefficient value'])
print(coefficients_df)

#### Inference with the model

In [ ]:
y_pred = lasso_rg.predict(X_test_pl)
y_pred

In [ ]:
results = pd.DataFrame({'Actual': y_test, 'Predicted': y_pred_pl})
print(results)

#### Evaluate the model

In [ ]:
mae = mean_absolute_error(y_test, y_pred_pl)
mse = mean_squared_error(y_test, y_pred_pl)
rmse = np.sqrt(mse)

print(f'Mean absolute error: {mae:.2f}')
print(f'Mean squared error: {mse:.2f}')
print(f'Root mean squared error: {rmse:.2f}')

In [ ]:
actual_minus_predicted = sum((y_test - y_pred_pl)**2)
actual_minus_actual_mean = sum((y_test - y_test.mean())**2)
r2 = 1 - actual_minus_predicted/actual_minus_actual_mean
print('R²:', r2)

In [ ]:
lasso_rg.score(X_test_pl, y_test)

In [ ]:
lasso_rg.score(X_train_pl, y_train)

## Linear / polynomial splines

In [ ]:
spline = SplineTransformer(n_knots=32, degree=1)

In [ ]:
X_train_sp = spline.fit_transform(X_train)
X_test_sp = spline.transform(X_test)

In [ ]:
feature_names_sp = spline.get_feature_names_out(input_features=X_train.columns)
feature_names_sp

In [ ]:
X_train.shape, X_train_sp.shape

In [ ]:
std_scaller = StandardScaler()
X_train_sp = std_scaller.fit_transform(X_train_sp)
X_test_sp = std_scaller.transform(X_test_sp)

#### Ridge

In [ ]:
ridge_rg = Ridge(alpha=36.08)
ridge_rg.fit(X_train_sp, y_train)

In [ ]:
ridge_rg.intercept_

In [ ]:
ridge_rg.coef_

In [ ]:
model_coefficients = ridge_rg.coef_

coefficients_df = pd.DataFrame(data = model_coefficients, 
                              index = feature_names_sp, 
                              columns = ['Coefficient value'])
print(coefficients_df)

#### Inference with the model

In [ ]:
y_pred_sp = ridge_rg.predict(X_test_sp)
y_pred_sp

In [ ]:
results = pd.DataFrame({'Actual': y_test, 'Predicted': y_pred_sp})
print(results)

#### Evaluate the model

In [ ]:
mae = mean_absolute_error(y_test, y_pred_sp)
mse = mean_squared_error(y_test, y_pred_sp)
rmse = np.sqrt(mse)

print(f'Mean absolute error: {mae:.2f}')
print(f'Mean squared error: {mse:.2f}')
print(f'Root mean squared error: {rmse:.2f}')

In [ ]:
R_2_test = ridge_rg.score(X_test_sp, y_test)
print(f'R² (test): {R_2_test}')

In [ ]:
R_2_train = ridge_rg.score(X_train_sp, y_train)
print(f'R² (train): {R_2_train}')